# Startegi : DOnt normalize

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import pathlib

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# !pip install tensorflow==2.0.0-alpha0
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import numpy as nay
print(tf.__version__)

2.0.0


In [2]:
path='data/train.csv'

In [3]:
df=pd.read_csv(path, index_col=0)
df=df[['x','y','z','x_sim','y_sim','z_sim']]
# df=df.drop('epoch', axis=1)
df.head()

,x,y,z,x_sim,y_sim,z_sim
id,,,,,,
0,-8855.823863,13117.780146,-20728.353233,-8843.131454,13138.221690,-20741.615306
1,-10567.672384,1619.746066,-24451.813271,-10555.500066,1649.289367,-24473.089556
2,-10578.684043,-10180.467460,-24238.280949,-10571.858472,-10145.939908,-24271.169776
3,-9148.251857,-20651.437460,-20720.381279,-9149.620794,-20618.200201,-20765.019094
4,-6719.092336,-28929.061629,-14938.907967,-6729.358857,-28902.271436,-14992.399986


In [4]:
mydata=df.copy()

In [5]:
mydata.isna().sum()

x        0
y        0
z        0
x_sim    0
y_sim    0
z_sim    0
dtype: int64

In [6]:
mydata = mydata.dropna()
mydata.shape

(649912, 6)

In [7]:
train_mydata = mydata.sample(frac=0.8,random_state=0)
test_mydata = mydata.drop(train_mydata.index)
test_mydata.head()

,x,y,z,x_sim,y_sim,z_sim
id,,,,,,
1,-10567.672384,1619.746066,-24451.813271,-10555.500066,1649.289367,-24473.089556
4,-6719.092336,-28929.061629,-14938.907967,-6729.358857,-28902.271436,-14992.399986
15,17281.246143,-2011.121378,39970.739873,17241.395285,-2016.770194,39958.759155
22,-1447.724638,28032.228987,-3918.192838,-1461.093061,28026.085525,-3959.767549
26,-10583.521489,-10145.003880,-24243.775722,-10574.016380,-10147.909472,-24270.149411


In [8]:
#a=train_mydata.shape
#b=test_mydata.shape
#print(a,b)
print(train_mydata.shape, test_mydata.shape)
train_mydata.head()

(519930, 6) (129982, 6)


,x,y,z,x_sim,y_sim,z_sim
id,,,,,,
162906,-22445.306373,5848.139687,805.906588,-21786.206588,7375.036638,391.178832
123662,-9086.781838,7654.307240,-6721.299832,-9545.479053,7482.025507,-6387.708079
901720,-33475.100342,-37591.455624,-3269.507831,-33263.131053,-37783.198553,-3617.165346
308669,11617.034853,2343.826977,-1821.565305,10718.558812,5174.769686,-1069.308627
1154759,25113.931453,-473.679010,41867.498593,19229.123354,12147.342145,31835.248429


In [15]:
label=['Vx','Vy','Vz','x','y','z'][3:]
train_stats = train_mydata.describe()
for i in label:
  train_stats.pop(i)
train_stats = train_stats.transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
x_sim,519930.0,-941.331006,25387.037253,-255037.952966,-13139.771199,-612.877803,11194.372299,200961.158000
y_sim,519930.0,319.853163,25603.245579,-281784.430040,-11758.892200,501.609913,12558.620911,209619.553857
z_sim,519930.0,-77.075471,20670.833347,-180250.922287,-7963.461284,57.093264,8253.028193,198526.091552


In [10]:
train_labels = nay.array([i for i in train_mydata[label].values])
test_labels = nay.array([i for i in test_mydata[label].values])

In [11]:
print(train_labels[0], test_labels[0])

[-22445.30637347   5848.13968719    805.90658772] [-10567.67238391   1619.74606635 -24451.81327102]


In [13]:
normed_train_data = train_mydata.drop(label, axis=1)
normed_test_data = test_mydata.drop(label, axis=1)

In [16]:
a=normed_test_data.describe()
a.transpose()

,count,mean,std,min,25%,50%,75%,max
x_sim,129982.0,-983.229404,25322.235545,-254874.999186,-13218.494933,-653.633569,11176.007672,200638.759019
y_sim,129982.0,343.373409,25624.462532,-282851.387908,-11649.250640,604.536998,12514.540291,209108.621952
z_sim,129982.0,-90.531541,20656.682202,-178436.101946,-7934.120403,55.969406,8173.489100,197982.276381


In [17]:
normed_test_data.keys()

Index(['x_sim', 'y_sim', 'z_sim'], dtype='object')

In [18]:
model=keras.Sequential()
model.add(layers.Dense(10, activation='relu', input_shape=[len(normed_train_data.keys())]))
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(3))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                40        
_________________________________________________________________
dense_1 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 33        
Total params: 183
Trainable params: 183
Non-trainable params: 0
_________________________________________________________________


In [19]:
optimizer = tf.keras.optimizers.RMSprop(0.001)
model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])

In [20]:
model.fit(normed_train_data, train_labels, validation_data=(normed_test_data, test_labels), epochs=10)

Train on 519930 samples, validate on 129982 samples
Epoch 1/10
519930/519930 [==============================] - 17s 32us/sample - loss: 55106704.9124 - mae: 3502.5347 - mse: 55106484.0000 - val_loss: 45215118.0579 - val_mae: 3187.4583 - val_mse: 45215104.0000
Epoch 2/10
519930/519930 [==============================] - 17s 32us/sample - loss: 45984251.7743 - mae: 3125.4290 - mse: 45984324.0000 - val_loss: 45179350.4186 - val_mae: 3114.9644 - val_mse: 45179376.0000
Epoch 3/10
519930/519930 [==============================] - 16s 31us/sample - loss: 45960630.5584 - mae: 3126.3164 - mse: 45960532.0000 - val_loss: 45138151.9014 - val_mae: 3126.4585 - val_mse: 45138092.0000
Epoch 4/10
519930/519930 [==============================] - 15s 29us/sample - loss: 45961500.5848 - mae: 3128.1914 - mse: 45961652.0000 - val_loss: 45394587.1939 - val_mae: 3052.7561 - val_mse: 45394596.0000
Epoch 5/10
519930/519930 [==============================] - 17s 33us/sample - loss: 45958672.0721 - mae: 3127.5012 -

In [25]:
test_predictions = model.predict(normed_train_data)

In [26]:
test_predictions

array([[-20358.213 ,   6864.3745,   1169.06  ],
       [ -8586.22  ,   6523.9907,  -6326.427 ],
       [-31493.857 , -37232.113 ,  -3881.0427],
       ...,
       [  6032.049 ,  -7238.6045, -18514.967 ],
       [  1746.001 ,  32188.562 , -42973.848 ],
       [-29767.805 , -33209.297 ,  -2136.344 ]], dtype=float32)

In [27]:
test_labels

array([[-10567.67238391,   1619.74606635, -24451.81327102],
       [ -6719.09233577, -28929.06162891, -14938.90796702],
       [ 17281.24614306,  -2011.12137796,  39970.73987322],
       ...,
       [  6809.32472235, -27094.76801954,    743.39273636],
       [  2071.20677085, -28307.11297851,   1753.73003927],
       [-23363.04479387,  -2906.07132041,   4747.2473861 ]])